# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import t as ta

"""
methods
"""

def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func=np.mean, mu=0, size=1):
    """Draw bootstrap replicates."""
    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)
    return bs_replicates


In [5]:
data = pd.io.stata.read_stata('./EDA_data/us_job_market_discrimination.dta')

In [6]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [7]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

1. What test is appropriate for this problem? Does CLT apply? 
For independent categorical data ???two 1-sample tests ???? will work to see if the two types of observations are relate
2. What are the null and alternate hypotheses? 
null: there is a relation between call backs and 'race', alternate: there is no relation


In [8]:
w = data[data.race=='w']
b = data[data.race=='b']

In [7]:
# Your solution to Q3 here


Two sample test

In [29]:
a = 0.05
#frequentist
bmu = np.mean(b["call"])
wmu = np.mean(w["call"])
b_s = np.std(b["call"])/len(b["call"])
w_s = np.std(w["call"])/len(w["call"])


# bay
bs_b = draw_bs_reps(b["call"], np.mean, size=10000)
bs_w = draw_bs_reps(w["call"], np.mean, size=10000)

# p value: if p < a then there is a significate difference
tt, pp = stats.ttest_ind(bs_w, bs_b)  
t_crit = ta.ppf(1 - 0.025, df)

# 0.95 confidence interval
cib = np.percentile(bs_b, [2.5, 97.5])
ciw = np.percentile(bs_w, [2.5, 97.5])

This statistics show that the mean of callbacks for 'white' sounding names will lie in an interval significantly higher that 'black' sounding names.

In [30]:
print("Is p < alpha? {}. conclusion: fail to reject ".format(pp < a)  + \
      "the null hypothesis which \nindicates that their is a signifigant difference in callbacks w/r/t 'race'. \n"+\
      "Is t > alpha? {}. conclusion: fail to reject the null hypothesis.".format(abs(tt) > a))
print("'black' CI: {} mean: {} \n'white' CI: {} mean: {}".format(cib, bmu, ciw, wmu))

Is p < alpha? True. conclusion: fail to reject the null hypothesis which 
indicates that their is a signifigant difference in callbacks w/r/t 'race'. 
Is t > alpha? True. conclusion: fail to reject the null hypothesis.
'black' CI: [ 0.0550308   0.07433265] mean: 0.0644763857126 
'white' CI: [ 0.08501027  0.10841889] mean: 0.0965092405677


In [28]:

# p value: if p < a then there is a significate difference
t, p = stats.ttest_ind(b["call"], w["call"])  

# 0.95 confidence interval
cib = np.percentile(b["call"], [2.5, 97.5])
ciw = np.percentile(w["call"], [2.5, 97.5])

In [ ]:
print("Is p < alpha? {}. conclusion: fail to reject ".format(p < a)  + \
      "the null hypothesis which \nindicates that their is a signifigant difference in callbacks w/r/t 'race'. \n"+\
      "Is t > alpha? {}. conclusion: fail to reject the null hypothesis.".format(abs(t) > a))
print("CI for b: {} \n CI for w: {}".format(cib,ciw))

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

Even though in this test 'race' sounding names have a significant impact on interview callbacks it does not mean that it is the deciding factor on if a candidate gets a call back for an interview. Apart of this dataset there are other features which could have contributed to an employers decision. Yet, we cannot reject that this feature is significant in said decision. 